# Proyecto II – Simulación Numérica del Atractor de Lorenz

## Introducción General

En el presente proyecto se abordará la simulación numérica del **atractor de Lorenz**, un sistema de ecuaciones diferenciales ordinarias (EDOs) de naturaleza no lineal que presenta comportamiento caótico. El objetivo es implementar desde cero dos métodos numéricos fundamentales para la resolución de sistemas dinámicos de primer orden:

- El **método de Euler**, por su sencillez computacional.
- El **método de Runge-Kutta de cuarto orden (RK4)**, reconocido por su precisión.

Ambos algoritmos serán utilizados para aproximar soluciones del sistema de Lorenz bajo distintas condiciones iniciales y parámetros, lo que permitirá observar y analizar el impacto de pequeñas variaciones en el comportamiento del sistema.

El sistema fue propuesto por **Edward Lorenz** en 1963 como un modelo simplificado de convección atmosférica. Su relevancia se debe a que constituye uno de los primeros ejemplos documentados de un sistema determinista caótico, donde mínimas perturbaciones en las condiciones iniciales provocan trayectorias divergentes a lo largo del tiempo, fenómeno conocido como el **efecto mariposa** [1].

El sistema está definido por el siguiente conjunto de ecuaciones diferenciales ordinarias:

$$
\begin{cases}
\dot{x} = \sigma (y - x) \\
\dot{y} = x(\rho - z) - y \\
\dot{z} = xy - \beta z
\end{cases}
$$

donde $\sigma$, $\rho$ y $\beta$ son parámetros positivos que caracterizan el sistema, y las variables $x(t)$, $y(t)$ y $z(t)$ representan funciones del tiempo.

Dado que no existe una solución analítica general para este sistema, será necesario utilizar métodos numéricos implementados manualmente en **Python**. Toda la lógica será construida desde cero utilizando únicamente herramientas como **NumPy** y **Matplotlib**.

---



# Sistema de Lorenz - Simulaciones

## 1. Definición del sistema de Lorenz

El sistema de Lorenz es un conjunto de ecuaciones diferenciales ordinarias no lineales de primer orden, derivado originalmente de un modelo simplificado para estudiar la convección térmica en la atmósfera. Fue propuesto por Edward Lorenz en 1963 y es conocido por su comportamiento caótico bajo ciertas condiciones iniciales y valores de parámetros [1].

Este sistema modela la evolución temporal de tres variables interdependientes, usualmente representadas como $x(t)$, $y(t)$ y $z(t)$, de acuerdo con el siguiente sistema:

$$
\begin{cases}
\dot{x} = \sigma (y - x) \\
\dot{y} = x(\rho - z) - y \\
\dot{z} = xy - \beta z
\end{cases}
$$

Donde:

- $\sigma$: número de Prandtl (relación entre difusión térmica y viscosidad),
- $\rho$: número de Rayleigh modificado (proporcional a la diferencia de temperatura),
- $\beta$: parámetro geométrico.

Cada una de estas ecuaciones describe la tasa de cambio de una variable con respecto al tiempo. El sistema puede escribirse de forma compacta como:

$$
\frac{d\vec{x}}{dt} = \vec{F}(\vec{x}, t)
$$

donde $\vec{x}(t) = [x(t), y(t), z(t)]^\top$ es el vector de estado y $\vec{F} = [\dot{x}, \dot{y}, \dot{z}]$ representa el **campo vectorial** que define la dinámica del sistema. Esta notación permite interpretar el comportamiento del sistema desde una perspectiva geométrica, facilitando el análisis de propiedades como la **divergencia** y el **rotacional**, fundamentales en el estudio de sistemas dinámicos tridimensionales [2].

A continuación, se implementa en Python una función que representa el sistema de Lorenz como un campo vectorial a partir de sus ecuaciones definidas.


In [1]:
import numpy as np

def lorenz_system(state, sigma, rho, beta):
    """
    Representación del sistema de Lorenz como campo vectorial.

    Parámetros:
    - state: vector de estado [x, y, z] en el tiempo actual.
    - sigma, rho, beta: parámetros del sistema de Lorenz.

    Retorna:
    - derivada: vector [dx/dt, dy/dt, dz/dt]
    """
    x, y, z = state
    dx = sigma * (y - x)
    dy = x * (rho - z) - y
    dz = x * y - beta * z
    return np.array([dx, dy, dz])


## 2. Implementación del Método de Euler

El **método de Euler** es uno de los algoritmos más simples para resolver numéricamente ecuaciones diferenciales ordinarias (EDOs). Consiste en aproximar la solución de una EDO de primer orden utilizando una expansión de **Taylor de primer grado** [3].

Sea una EDO en forma vectorial:

$$
\frac{d\vec{x}}{dt} = \vec{F}(\vec{x}, t), \quad \vec{x}(t_0) = \vec{x}_0
$$

donde $\vec{x}(t)$ representa el vector de estado del sistema en el instante $t$, y $\vec{F}$ es una función que describe su dinámica.

La idea principal del método es utilizar el valor actual de la derivada para proyectar la solución hacia adelante en el tiempo. Si se desea calcular una aproximación en el instante $t_{n+1} = t_n + h$, se emplea la expansión de Taylor:

$$
\vec{x}(t_{n+1}) = \vec{x}(t_n) + h \cdot \frac{d\vec{x}}{dt}(t_n) + \mathcal{O}(h^2)
$$

Despreciando el término de orden superior $\mathcal{O}(h^2)$, se obtiene la **fórmula de Euler**:

$$
\vec{x}_{n+1} = \vec{x}_n + h \cdot \vec{F}(\vec{x}_n, t_n)
$$

donde:
- $\vec{x}_n$ es la aproximación de la solución en el instante $t_n$,
- $h$ es el tamaño del paso temporal.

Este procedimiento se repite de forma iterativa para obtener la trayectoria aproximada del sistema entre un tiempo inicial $t_0$ y uno final $t_f$. La calidad de la solución depende fuertemente del tamaño del paso $h$: valores grandes pueden generar errores significativos, mientras que pasos pequeños aumentan la precisión a costa de mayor tiempo de cómputo.

### Aplicación al sistema de Lorenz

El sistema de Lorenz, al ser un sistema tridimensional de EDOs no lineales, se adapta directamente al esquema anterior. En este caso, el vector $\vec{x}$ corresponde a:

$$
\vec{x}(t) = [x(t),\; y(t),\; z(t)]^\top
$$

y el campo vectorial $\vec{F}$ está definido por las ecuaciones:

$$
\vec{F}(\vec{x}) =
\begin{bmatrix}
\sigma (y - x) \\
x (\rho - z) - y \\
xy - \beta z
\end{bmatrix}
$$

A continuación, se presenta la implementación computacional del método de Euler para sistemas de EDOs de este tipo, de forma general y reutilizable.


In [2]:
def euler_method(f, x0, t0, tf, h, args=()):
    """
    Implementación del método de Euler para sistemas de EDOs.

    Parámetros:
    - f: función que define el sistema (e.g., lorenz_system)
    - x0: vector de condiciones iniciales [x0, y0, z0]
    - t0: tiempo inicial
    - tf: tiempo final
    - h: paso de integración
    - args: tupla de parámetros adicionales para el sistema (e.g., sigma, rho, beta)

    Retorna:
    - t_values: arreglo de tiempos
    - x_values: arreglo de soluciones aproximadas en cada paso
    """
    num_steps = int((tf - t0) / h)
    t_values = np.linspace(t0, tf, num_steps + 1)
    x_values = np.zeros((num_steps + 1, len(x0)))
    x_values[0] = x0

    for i in range(num_steps):
        x_values[i + 1] = x_values[i] + h * f(x_values[i], *args)

    return t_values, x_values


## 3. Implementación del Método de Runge-Kutta de Cuarto Orden (RK4)

El **método de Runge-Kutta de cuarto orden (RK4)** es uno de los algoritmos más utilizados para la resolución numérica de ecuaciones diferenciales ordinarias debido a su buen equilibrio entre **precisión** y **eficiencia computacional**. A diferencia del método de Euler, que utiliza una única evaluación del campo vectorial $\vec{F}$ por paso, RK4 realiza **cuatro evaluaciones por paso**, combinándolas ponderadamente para obtener una estimación más precisa del siguiente valor [4].

Para un sistema de EDOs expresado en forma general:

$$
\frac{d\vec{x}}{dt} = \vec{F}(\vec{x}, t), \quad \vec{x}(t_0) = \vec{x}_0
$$

el método RK4 avanza desde $\vec{x}_n$ hasta $\vec{x}_{n+1}$ usando las siguientes fórmulas:

\begin{aligned}
k_1 &= \vec{F}(\vec{x}_n, t_n) \\
k_2 &= \vec{F}\left(\vec{x}_n + \frac{h}{2}k_1,\; t_n + \frac{h}{2}\right) \\
k_3 &= \vec{F}\left(\vec{x}_n + \frac{h}{2}k_2,\; t_n + \frac{h}{2}\right) \\
k_4 &= \vec{F}\left(\vec{x}_n + h \cdot k_3,\; t_n + h\right) \\
\vec{x}_{n+1} &= \vec{x}_n + \frac{h}{6}(k_1 + 2k_2 + 2k_3 + k_4)
\end{aligned}

Donde:
- $h$ es el tamaño del paso temporal,
- $k_1$, $k_2$, $k_3$, $k_4$ son **estimaciones intermedias** de la derivada.

El método RK4 logra un **error local de orden $h^5$** y un **error global de orden $h^4$**, lo que significa que es mucho más preciso que Euler (orden 1) al mismo tamaño de paso. Esta característica lo hace especialmente adecuado para sistemas con comportamientos caóticos o sensibles, como el atractor de Lorenz.

### Aplicación al sistema de Lorenz

El sistema de Lorenz puede resolverse numéricamente utilizando este método, aplicando el vector de estado $\vec{x} = [x, y, z]^\top$ y el campo vectorial no lineal previamente definido. El uso de RK4 permite capturar con mayor fidelidad las trayectorias del sistema, evitando errores acumulados y oscilaciones artificiales producidas por métodos de menor orden.

A continuación se implementa la versión general del método RK4 para sistemas tridimensionales como el de Lorenz.

In [3]:
def runge_kutta_4(f, x0, t0, tf, h, args=()):
    """
    Implementación del método de Runge-Kutta de cuarto orden (RK4) para sistemas de EDOs.

    Parámetros:
    - f: función que define el sistema (e.g., lorenz_system)
    - x0: vector de condiciones iniciales [x0, y0, z0]
    - t0: tiempo inicial
    - tf: tiempo final
    - h: paso de integración
    - args: tupla de parámetros adicionales para el sistema (e.g., sigma, rho, beta)

    Retorna:
    - t_values: arreglo de tiempos
    - x_values: arreglo de soluciones aproximadas en cada paso
    """
    num_steps = int((tf - t0) / h)
    t_values = np.linspace(t0, tf, num_steps + 1)
    x_values = np.zeros((num_steps + 1, len(x0)))
    x_values[0] = x0

    for i in range(num_steps):
        t = t_values[i]
        x = x_values[i]

        k1 = f(x, *args)
        k2 = f(x + 0.5 * h * k1, *args)
        k3 = f(x + 0.5 * h * k2, *args)
        k4 = f(x + h * k3, *args)

        x_values[i + 1] = x + (h / 6.0) * (k1 + 2*k2 + 2*k3 + k4)

    return t_values, x_values
